<a href="https://colab.research.google.com/github/LeonChou5311/LeonChou5311/blob/main/BB_LDA_Web_Testing_version_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run in python console
!pip install nltk
!pip install pyLDAvis
!pip install scipy
!pip install boto

import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from','bad','subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
stop_words.extend(['of','for','and','day','due','put','beg','mfm','fbr','cra','uav','etc','far','sem','iot','tsa','one','dna','fix','new','art','set','maf','ice','yet','jam','age','auc','non','job','big','low','way','cpu','mhz','bit','eye','sub','map','key','lie','lab','rct','end','web','aim','act'])
stop_words.extend(['wsis','pbpm','thus','sc','lrmb','lp','uavs','give','user','long','work','sedc','shapc','rr'])
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

     |████████████████████████████████| 1.6MB 2.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=9753f6397d46ea35d507152ea5d3ba99865d8e7c40770c7c2b4d613326973830
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis
     |████████████████████████████████| 1.4MB 2.9MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import os, glob
import pandas as pd
import re
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


# use Pandas to read 2.csv
import pandas as pd
Web = pd.read_csv('Web.csv')
print(Web)

Saving Web.csv to Web.csv
User uploaded file "Web.csv" with length 406597 bytes
    Publication Type  ... Unnamed: 67
0                  J  ...         NaN
1                  J  ...         NaN
2                  J  ...         NaN
3                  J  ...         NaN
4                  J  ...         NaN
..               ...  ...         ...
121                J  ...         NaN
122                J  ...         NaN
123                J  ...         NaN
124                B  ...         NaN
125                B  ...         NaN

[126 rows x 68 columns]


**Web-Step 1. Import csv files**

In [ ]:
# Web-Step 1. Import csv files

In [ ]:
#Step 2. Select the columns ('Author Keywords', 'Article Title', 'Source Title' and renname)

In [ ]:
Web_re = Web.rename(columns= {'Author Keywords': 'Author_Keywords', 'Document Title': 'Document_Title', 'Article Title': 'Article_Title'}, inplace = False)

In [ ]:
Web_re['corpus'] = Web_re['Article_Title'] + ' ' + Web_re['Author_Keywords'] + ' ' + Web_re['Abstract']
Web_select = Web_re['corpus']

In [ ]:
Web_drop_nan = Web_select[Web_select.notna()]
print(Web_drop_nan)

0      A comparison of instance-level counterfactual ...
1      A proposed collaborative framework by using ar...
3      Artificial intelligence from A to Z: From neur...
5      The uncertainty and explainability in object r...
6      A kind of artificial intelligence model based ...
                             ...                        
120    Combining FDI and AI approaches within causal-...
121    The chinese room argument reconsidered: Essent...
122    Intrinsic cognitive models philosophy; artific...
123    Wittgenstein's deflationary account of referen...
124    Application of artificial intelligence approac...
Name: corpus, Length: 102, dtype: object


In [ ]:
print(Web_drop_nan)

0      A comparison of instance-level counterfactual ...
1      A proposed collaborative framework by using ar...
3      Artificial intelligence from A to Z: From neur...
5      The uncertainty and explainability in object r...
6      A kind of artificial intelligence model based ...
                             ...                        
120    Combining FDI and AI approaches within causal-...
121    The chinese room argument reconsidered: Essent...
122    Intrinsic cognitive models philosophy; artific...
123    Wittgenstein's deflationary account of referen...
124    Application of artificial intelligence approac...
Name: corpus, Length: 102, dtype: object


In [ ]:
Web_drop_nan

0      A comparison of instance-level counterfactual ...
1      A proposed collaborative framework by using ar...
3      Artificial intelligence from A to Z: From neur...
5      The uncertainty and explainability in object r...
6      A kind of artificial intelligence model based ...
                             ...                        
120    Combining FDI and AI approaches within causal-...
121    The chinese room argument reconsidered: Essent...
122    Intrinsic cognitive models philosophy; artific...
123    Wittgenstein's deflationary account of referen...
124    Application of artificial intelligence approac...
Name: corpus, Length: 102, dtype: object

In [ ]:
Web_list = Web_drop_nan.tolist()
print(Web_list)

['A comparison of instance-level counterfactual explanation algorithms for behavioral and textual data: SEDC, LIME-C and SHAP-C Comparative study; Counterfactual explanations; Instance-level explanations; Explainable artificial intelligence; Explanation algorithms; Binary classification; Behavioral data; Textual data Predictive systems based on high-dimensional behavioral and textual data have serious comprehensibility and transparency issues: linear models require investigating thousands of coefficients, while the opaqueness of nonlinear models makes things worse. Counterfactual explanations are becoming increasingly popular for generating insight into model predictions. This study aligns the recently proposed linear interpretable model-agnostic explainer and Shapley additive explanations with the notion of counterfactual explanations, and empirically compares the effectiveness and efficiency of these novel algorithms against a model-agnostic heuristic search algorithm for finding evi

In [ ]:
#Cleaning up text

In [ ]:
 #Remove Emails
data = []
for text in Web_list:
    
    # put process to lower case
    process = text.lower()
    process = process.replace('"', '')
    process = process.replace("_", " ")
    process = process.replace("causality","cause")
    process = process.replace("causation","cause")
    process = process.replace("caused","cause")
    process = process.replace("explainable","explain")
    process = process.replace("explains","explain")
    process = process.replace("explaind","explain")
    process = process.replace("explanation","explain")
    process = process.replace("interpretable","interpret")
    process = process.replace("inrepretation","interpret")
    process = process.replace("derailment","")
    process = process.replace("data", " ")
    process = process.replace('rule', '')
    process = process.replace("study", " ")
    process = process.replace('studying', '')
    process = process.replace("student", " ")
    process = process.replace('also', '')
    process = process.replace("paper", " ")
    process = process.replace("set", " ")
    process = process.replace('based', '')
    process = process.replace("result", " ")
    process = process.replace("©", " ")
    process = process.replace("elsevier", " ")
    process = process.replace("bv", " ")
    process = re.sub('\S*@\S*\s?', '', process)
    process = re.sub("\'", '', process)
    process = re.sub("[\d-]", '', process)
    process = re.sub('https?:\/\/.*[\r\n]*', '', process)
    process = re.sub('\.|:|;', '', process)
    process = re.sub('\([a-z]\)', '', process)
    process = re.sub('\(|\)', '', process)
    process = re.sub('<', '', process)
    process = re.sub('\s+', ' ', process)
    process = re.sub('url', '', process)
    process = re.sub('\*|\+', '', process)
    process = re.sub('\[|\]|\"|\'|/''|\_','', process)
    
    
    data.append( process )
print(data)

['a comparison of instancelevel counterfactual explain algorithms for behavioral and textual sedc, limec and shapc comparative counterfactual explains instancelevel explains explain artificial intelligence explain algorithms binary classification behavioral textual predictive systems on highdimensional behavioral and textual have serious comprehensibility and transparency issues linear models require investigating thousands of coefficients, while the opaqueness of nonlinear models makes things worse counterfactual explains are becoming increasingly popular for generating insight into model predictions this aligns the recently proposed linear interpret modelagnostic explainer and shapley additive explains with the notion of counterfactual explains, and empirically compares the effectiveness and efficiency of these novel algorithms against a modelagnostic heuristic search algorithm for finding evidence counterfactuals using behavioral and textual s we show that different search methods h

In [ ]:
#Step6. Tokenize words

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def sent_to_words(sentences):
                
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

#Web_drop_Abstract_list_tokenize = list(sent_to_words(data))

#print(Web_drop_Abstract_list_tokenize)

tokenized_models = [word_tokenize(str(i)) for i in data]
tokenized_models

In [ ]:
#Remove stopword

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import csv

stopset = set(stopwords.words('english'))

Web_nostops = []
for m in tokenized_models:
    stop_m = []
    for i in m:
        if (str(i).lower() not in stopset) and (len(str(i)) > 3):
            stop_m.append(str(i))
    Web_nostops.append(stop_m)

#print(Web_nostops)
stop_models = [i for i in tokenized_models if str(i).lower() not in stopset]
print('token:'+str(stop_models))

token:[['a', 'comparison', 'of', 'instancelevel', 'counterfactual', 'explain', 'algorithms', 'for', 'behavioral', 'and', 'textual', 'sedc', ',', 'limec', 'and', 'shapc', 'comparative', 'counterfactual', 'explains', 'instancelevel', 'explains', 'explain', 'artificial', 'intelligence', 'explain', 'algorithms', 'binary', 'classification', 'behavioral', 'textual', 'predictive', 'systems', 'on', 'highdimensional', 'behavioral', 'and', 'textual', 'have', 'serious', 'comprehensibility', 'and', 'transparency', 'issues', 'linear', 'models', 'require', 'investigating', 'thousands', 'of', 'coefficients', ',', 'while', 'the', 'opaqueness', 'of', 'nonlinear', 'models', 'makes', 'things', 'worse', 'counterfactual', 'explains', 'are', 'becoming', 'increasingly', 'popular', 'for', 'generating', 'insight', 'into', 'model', 'predictions', 'this', 'aligns', 'the', 'recently', 'proposed', 'linear', 'interpret', 'modelagnostic', 'explainer', 'and', 'shapley', 'additive', 'explains', 'with', 'the', 'notion'

In [ ]:
#Step7. Creating Bigram and Trigram models

In [ ]:

def make_bigrams(Web_nostops):
    bigram = gensim.models.Phrases(Web_nostops, min_count=5, threshold=100) # higher threshold fewer phrases.
    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return [bigram_mod[doc] for doc in Web_nostops]

def make_trigrams(Web_nostops):
    trigram = gensim.models.Phrases(bigram[Web_nostops], threshold=100)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return [trigram_mod[bigram_mod[doc]] for doc in Web_nostops]


In [ ]:
# Form Bigrams
Web_bigrams = make_bigrams(Web_nostops)

Web_bigrams
    

[['comparison',
  'instancelevel',
  'counterfactual',
  'explain',
  'algorithms',
  'behavioral',
  'textual',
  'sedc',
  'limec',
  'shapc',
  'comparative',
  'counterfactual',
  'explains',
  'instancelevel',
  'explains',
  'explain',
  'artificial',
  'intelligence',
  'explain',
  'algorithms',
  'binary',
  'classification',
  'behavioral',
  'textual',
  'predictive',
  'systems',
  'highdimensional',
  'behavioral',
  'textual',
  'serious',
  'comprehensibility',
  'transparency',
  'issues',
  'linear',
  'models',
  'require',
  'investigating',
  'thousands',
  'coefficients',
  'opaqueness',
  'nonlinear',
  'models',
  'makes',
  'things',
  'worse',
  'counterfactual',
  'explains',
  'becoming',
  'increasingly',
  'popular',
  'generating',
  'insight',
  'model',
  'predictions',
  'aligns',
  'recently',
  'proposed',
  'linear',
  'interpret',
  'modelagnostic',
  'explainer',
  'shapley',
  'additive',
  'explains',
  'notion',
  'counterfactual',
  'explains',

In [ ]:
#Lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
lemmatized_bigrams = []

for tokens in Web_bigrams:
    temp_lemma = []
    for token in tokens:
        temp_lemma.append( lemmatizer.lemmatize(token) )
    lemmatized_bigrams.append(temp_lemma)

print(lemmatized_bigrams)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[['comparison', 'instancelevel', 'counterfactual', 'explain', 'algorithm', 'behavioral', 'textual', 'sedc', 'limec', 'shapc', 'comparative', 'counterfactual', 'explains', 'instancelevel', 'explains', 'explain', 'artificial', 'intelligence', 'explain', 'algorithm', 'binary', 'classification', 'behavioral', 'textual', 'predictive', 'system', 'highdimensional', 'behavioral', 'textual', 'serious', 'comprehensibility', 'transparency', 'issue', 'linear', 'model', 'require', 'investigating', 'thousand', 'coefficient', 'opaqueness', 'nonlinear', 'model', 'make', 'thing', 'worse', 'counterfactual', 'explains', 'becoming', 'increasingly', 'popular', 'generating', 'insight', 'model', 'prediction', 'aligns', 'recently', 'proposed', 'linear', 'interpret', 'modelagnostic', 'explainer', 'shapley', 'additive', 'explains', 'notion', 'counterfactual', 'explains', 'empirically', 'compare', 'effectiv

In [ ]:
#from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer


#ps = PorterStemmer()
#ps = LancasterStemmer()

#stemmed_bigrams = []
#for tokens in lemmatized_bigrams:
#    temp_stemmed = []
#    for token in tokens:
#        temp_stemmed.append( ps.stem(token) )
#    stemmed_bigrams.append(temp_stemmed)

#print(stemmed_bigrams)


In [ ]:
#from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer


#ps = PorterStemmer()
#ps = LancasterStemmer()

#stemmed_bigrams = []
#for tokens in lemmatized_bigrams:
#    temp_stemmed = []
#    for token in tokens:
#        temp_stemmed.append( ps.stem(token) )
#    stemmed_bigrams.append(temp_stemmed)

#print(stemmed_bigrams)


In [ ]:
 texts = lemmatized_bigrams
#texts = stemmed_bigrams
print(texts)

[['comparison', 'instancelevel', 'counterfactual', 'explain', 'algorithm', 'behavioral', 'textual', 'sedc', 'limec', 'shapc', 'comparative', 'counterfactual', 'explains', 'instancelevel', 'explains', 'explain', 'artificial', 'intelligence', 'explain', 'algorithm', 'binary', 'classification', 'behavioral', 'textual', 'predictive', 'system', 'highdimensional', 'behavioral', 'textual', 'serious', 'comprehensibility', 'transparency', 'issue', 'linear', 'model', 'require', 'investigating', 'thousand', 'coefficient', 'opaqueness', 'nonlinear', 'model', 'make', 'thing', 'worse', 'counterfactual', 'explains', 'becoming', 'increasingly', 'popular', 'generating', 'insight', 'model', 'prediction', 'aligns', 'recently', 'proposed', 'linear', 'interpret', 'modelagnostic', 'explainer', 'shapley', 'additive', 'explains', 'notion', 'counterfactual', 'explains', 'empirically', 'compare', 'effectiveness', 'efficiency', 'novel', 'algorithm', 'modelagnostic', 'heuristic', 'search', 'algorithm', 'finding',

In [ ]:
#Step9. Create the Dictionary and Corpus needed for Topic modeling

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(texts)

# Create Corpus
#texts = lemmatized_bigrams
#texts = stemmed_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
#merge casual/ cause etc....
# View
print(corpus[:6])

[[(0, 1), (1, 4), (2, 1), (3, 1), (4, 1), (5, 4), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 4), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 3), (21, 1), (22, 5), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 3), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 4), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1)], [(3, 2), (24, 1), (32, 1), (35, 2), (47, 1), (50, 1), (53, 2), (64, 2), (67, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 5), (77, 1), (78, 1), (79, 3), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 4), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 5), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 

In [ ]:
id2word[10]

'compare'

In [ ]:
## Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:10]]

[[('additive', 1),
  ('algorithm', 4),
  ('aligns', 1),
  ('artificial', 1),
  ('becoming', 1),
  ('behavioral', 4),
  ('binary', 1),
  ('classification', 1),
  ('coefficient', 1),
  ('comparative', 1),
  ('compare', 1),
  ('comparison', 1),
  ('comprehensibility', 1),
  ('counterfactual', 4),
  ('counterfactuals', 1),
  ('different', 2),
  ('effectiveness', 1),
  ('efficiency', 1),
  ('empirically', 1),
  ('evidence', 1),
  ('explain', 3),
  ('explainer', 1),
  ('explains', 5),
  ('finding', 1),
  ('future', 1),
  ('generating', 1),
  ('heuristic', 1),
  ('highdimensional', 1),
  ('importantly', 1),
  ('increasingly', 1),
  ('insight', 1),
  ('instancelevel', 2),
  ('intelligence', 1),
  ('interpret', 1),
  ('investigating', 1),
  ('issue', 1),
  ('limec', 1),
  ('linear', 2),
  ('make', 1),
  ('method', 1),
  ('model', 3),
  ('modelagnostic', 2),
  ('much', 1),
  ('nonlinear', 1),
  ('notion', 1),
  ('novel', 1),
  ('opaqueness', 1),
  ('popular', 1),
  ('prediction', 1),
  ('predict

In [ ]:
#Step 10. Compute the Optimum Number of Topics: LDA-Coherence Score

In [ ]:
#Build LDA Topic model

In [ ]:
  # Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=500,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
#Step11. View the topic in LDA model

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())

[(0,
  '0.038*"intelligence" + 0.030*"accident" + 0.024*"method" + 0.024*"analysis" '
  '+ 0.023*"artificial" + 0.023*"fuzzy_logic" + 0.016*"system" + '
  '0.015*"safety" + 0.015*"factor" + 0.015*"intelligent"'),
 (1,
  '0.011*"system" + 0.011*"intelligence" + 0.010*"artificial" + 0.008*"model" '
  '+ 0.008*"explain" + 0.006*"cause" + 0.005*"causal" + 0.005*"algorithm" + '
  '0.005*"human" + 0.004*"method"'),
 (2,
  '0.031*"reference" + 0.019*"account" + 0.019*"press" + 0.019*"cambridge" + '
  '0.019*"intentionality" + 0.019*"wittgenstein" + 0.013*"science" + '
  '0.013*"representation" + 0.013*"thought" + 0.013*"mental"')]


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.916587713206747

Coherence Score:  0.5279385166134847


In [ ]:
#Step12. Visualize the topics-keywords

In [ ]:
#!pip install pyLDAvis

In [ ]:
# Visualize the topics
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.060964 -0.147617       1        1  93.844855
0     -0.166022  0.108157       2        1   4.176764
2      0.226986  0.039461       3        1   1.978381, topic_info=              Term        Freq       Total Category  logprob  loglift
32    intelligence  166.000000  166.000000  Default  30.0000  30.0000
3       artificial  149.000000  149.000000  Default  29.0000  29.0000
62          system  156.000000  156.000000  Default  28.0000  28.0000
39          method   68.000000   68.000000  Default  27.0000  27.0000
74        analysis   62.000000   62.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
202       category    1.794341    3.625464   Topic3  -5.0405   3.2195
249     psychology    1.841671    8.383463   Topic3  -5.0145   2.4073
205      cognitive    1.905633   37.301952   Topic3  -4.9803   0.9487
2296    hypothesis    1.823458    7.140414   Topic3  -5.0244   2.5579
588         theory    1.843401   34.603636   Topic3  -5.0135   0.9905

[176 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
507       1  0.711454          able
507       2  0.323388          able
1526      1  0.054958      accident
1526      2  0.989249      accident
408       1  0.284392       account
...     ...       ...           ...
3519      1  0.172244  wittgenstein
3519      3  0.861220  wittgenstein
157       1  0.998067          work
1075      1  0.916610         world
1075      3  0.107836         world

[258 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

In [ ]:
#Step13. What is the Dominant topic and its percentage contribution in each document

In [ ]:
Web_drop_nan.iloc[1]

'A proposed collaborative framework by using artificial intelligence-internet of things (AI-IoT) in COVID-19 pandemic situation for healthcare workers Artificial intelligence; COVID-19; epidemic outbreak; health care worker; Internet of things (IoT) The COVID-19 epidemic has created a global fear for humanity. Despite technological advancement, the use of face masks, hand gloves, and sanitizers are the only available preventives measures to stop the spread and transmission of coronavirus. Healthcare providers and medical staffs are always at risk of infection as they deal with the infected person at the frontline. The safety of health care workers is essential as the number of infected health care works is increasing day by day. A detailed literature review is done on a previous epidemic outbreak and COVID-19 by using suitable keywords on SCOPUS. During the analysis of literature, several challenges were found that restrict the operation of the health care workers and cause safety issu

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()
    #sent_topics_df = texts

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,1.0,0.9993,"system, intelligence, artificial, model, expla...","[comparison, instancelevel, counterfactual, ex..."
1,1,1.0,0.9995,"system, intelligence, artificial, model, expla...","[proposed, collaborative, framework, using, ar..."
2,2,1.0,0.9990,"system, intelligence, artificial, model, expla...","[artificial, intelligence, neural_network, leg..."
3,3,1.0,0.9534,"system, intelligence, artificial, model, expla...","[uncertainty, explainability, object, recognit..."
4,4,1.0,0.9995,"system, intelligence, artificial, model, expla...","[kind, artificial, intelligence, model, nature..."
...,...,...,...,...,...
97,97,1.0,0.9995,"system, intelligence, artificial, model, expla...","[combining, approach, within, causalmodel, dia..."
98,98,2.0,0.7162,"reference, account, press, cambridge, intentio...","[chinese, room, argument, reconsidered, essent..."
99,99,1.0,0.7947,"system, intelligence, artificial, model, expla...","[intrinsic, cognitive, model, philosophy, arti..."
100,100,2.0,0.9860,"reference, account, press, cambridge, intentio...","[wittgenstein, deflationary, account, referenc..."


In [ ]:
Web_droplicate = df_dominant_topic.drop_duplicates(subset= ['Document_No'])
print(Web_droplicate)

     Document_No  ...                                               Text
0              0  ...  [comparison, instancelevel, counterfactual, ex...
1              1  ...  [proposed, collaborative, framework, using, ar...
2              2  ...  [artificial, intelligence, neural_network, leg...
3              3  ...  [uncertainty, explainability, object, recognit...
4              4  ...  [kind, artificial, intelligence, model, nature...
..           ...  ...                                                ...
97            97  ...  [combining, approach, within, causalmodel, dia...
98            98  ...  [chinese, room, argument, reconsidered, essent...
99            99  ...  [intrinsic, cognitive, model, philosophy, arti...
100          100  ...  [wittgenstein, deflationary, account, referenc...
101          101  ...  [application, artificial, intelligence, approa...

[102 rows x 5 columns]


In [ ]:
Web_topic_select = Web_droplicate[(Web_droplicate.Dominant_Topic == 1.0)]
print(Web_topic_select)

    Document_No  ...                                               Text
0             0  ...  [comparison, instancelevel, counterfactual, ex...
1             1  ...  [proposed, collaborative, framework, using, ar...
2             2  ...  [artificial, intelligence, neural_network, leg...
3             3  ...  [uncertainty, explainability, object, recognit...
4             4  ...  [kind, artificial, intelligence, model, nature...
..          ...  ...                                                ...
94           94  ...  [functionbehaviorstate, approach, designing, h...
95           95  ...  [synthesis, blind, source, separation, algorit...
96           96  ...  [machine, know, philosophical, problem, roboti...
97           97  ...  [combining, approach, within, causalmodel, dia...
99           99  ...  [intrinsic, cognitive, model, philosophy, arti...

[99 rows x 5 columns]


In [ ]:
#Remove nan duplicate
Web_nan_droplicate = Web_drop_nan.drop_duplicates()
print(Web_nan_droplicate)

0      A comparison of instance-level counterfactual ...
1      A proposed collaborative framework by using ar...
3      Artificial intelligence from A to Z: From neur...
5      The uncertainty and explainability in object r...
6      A kind of artificial intelligence model based ...
                             ...                        
120    Combining FDI and AI approaches within causal-...
121    The chinese room argument reconsidered: Essent...
122    Intrinsic cognitive models philosophy; artific...
123    Wittgenstein's deflationary account of referen...
124    Application of artificial intelligence approac...
Name: corpus, Length: 102, dtype: object


In [ ]:
counter_causability = 0
counter_causal = 0
counter_counterf = 0
counter_interpret = 0
counter_explain = 0
counter_reasoning = 0
counter_bayesian = 0

for t, d in zip(Web_droplicate[Web_droplicate.Dominant_Topic == 1.0].Text.values, df_dominant_topic.Document_No ):
  if( ('causability' in t)):
    counter_causability = counter_causability + 1

  if( ('causal' in t) ):
    print(d)
    print(t)
    counter_causal = counter_causal + 1

  if( ('counterfactual' in t) ):
    counter_counterf = counter_counterf + 1

  if( ('interpret' in t) ):
    print(d)
    print(t)
    counter_interpret = counter_interpret + 1

  if( ('explain' in t)):
    counter_explain = counter_explain + 1
 
  if( ('reasoning' in t) ):

    counter_reasoning = counter_reasoning + 1

  if( ('bayesian' in t)):
    counter_bayesian = counter_bayesian + 1

    #print(t)


print(counter_causability, 'causability')
print(counter_causal, 'causal')
print(counter_counterf, 'counterfactual')
print(counter_interpret, 'interpret')
print(counter_explain, 'explain')
print(counter_reasoning, 'reasoning')
print(counter_bayesian, 'bayesian')

0
['comparison', 'instancelevel', 'counterfactual', 'explain', 'algorithm', 'behavioral', 'textual', 'sedc', 'limec', 'shapc', 'comparative', 'counterfactual', 'explains', 'instancelevel', 'explains', 'explain', 'artificial', 'intelligence', 'explain', 'algorithm', 'binary', 'classification', 'behavioral', 'textual', 'predictive', 'system', 'highdimensional', 'behavioral', 'textual', 'serious', 'comprehensibility', 'transparency', 'issue', 'linear', 'model', 'require', 'investigating', 'thousand', 'coefficient', 'opaqueness', 'nonlinear', 'model', 'make', 'thing', 'worse', 'counterfactual', 'explains', 'becoming', 'increasingly', 'popular', 'generating', 'insight', 'model', 'prediction', 'aligns', 'recently', 'proposed', 'linear', 'interpret', 'modelagnostic', 'explainer', 'shapley', 'additive', 'explains', 'notion', 'counterfactual', 'explains', 'empirically', 'compare', 'effectiveness', 'efficiency', 'novel', 'algorithm', 'modelagnostic', 'heuristic', 'search', 'algorithm', 'finding'

In [ ]:
from google.colab import files
Web_topic_select.to_csv(r'Web_topic_select.csv', index = False)
files.download("Web_topic_select.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

Web_nan_droplicate.to_csv(r'Web_nan_droplicate.csv', index = False)
files.download("Web_nan_droplicate.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>